In [28]:
from langchain_community.document_loaders import PyPDFLoader
import os

path = r'D:\Programming\Summarize-PDF'
documents = []
for pdf_file in os.listdir('D:/Programming/Summarize-PDF'):
    #print(pdf_file)
    if pdf_file.endswith(".pdf"):
        #print(pdf_file)
        pdf_path_join = os.path.join(path,pdf_file)
        loader = PyPDFLoader(pdf_path_join)
        for doc in loader.lazy_load():
            #print(doc)
            documents.append(doc)
            
        
print(documents)

[Document(metadata={'source': 'D:\\Programming\\Summarize-PDF\\IRT_Machine_Translation.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/320730405\nMachine Translation\nPreprint · October 2017\nCITATIONS\n0\nREADS\n38,236\n1 author:\nMuhammad Irfan\nBahria University\n1 PUBLICATION\xa0\xa0\xa00 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Muhammad Irfan on 31 October 2017.\nThe user has requested enhancement of the downloaded file.'), Document(metadata={'source': 'D:\\Programming\\Summarize-PDF\\IRT_Machine_Translation.pdf', 'page': 1}, page_content='Machine Translation\nMuhammad Irfan\nDepartment of Computer Science\nBahria University Islamabad\nEmail: muhammad.Irfan@outlook.com\nAbstract—Language is core medium of communication and\ntranslation is core tool for the understand the information\nin unknown language.Machine translation helps the people to\nun

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
chunk = RecursiveCharacterTextSplitter( chunk_size = 1000, chunk_overlap = False)
split_pdf = chunk.split_documents(documents)
len(split_pdf)

37

In [30]:
split_pdf

[Document(metadata={'source': 'D:\\Programming\\Summarize-PDF\\IRT_Machine_Translation.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/320730405\nMachine Translation\nPreprint · October 2017\nCITATIONS\n0\nREADS\n38,236\n1 author:\nMuhammad Irfan\nBahria University\n1 PUBLICATION\xa0\xa0\xa00 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Muhammad Irfan on 31 October 2017.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={'source': 'D:\\Programming\\Summarize-PDF\\IRT_Machine_Translation.pdf', 'page': 1}, page_content='Machine Translation\nMuhammad Irfan\nDepartment of Computer Science\nBahria University Islamabad\nEmail: muhammad.Irfan@outlook.com\nAbstract—Language is core medium of communication and\ntranslation is core tool for the understand the information\nin unknown language.Machine translation helps the people to\nu

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(documents=split_pdf, embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
retriever = db.as_retriever()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14388\1441196560.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  db = FAISS.from_documents(documents=split_pdf, embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

model = ChatGroq(temperature = 0.1, model_name = "llama-3.3-70b-versatile")

In [32]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a PDF Reader Assistant."
    "Provide concise and professional responses."
    "If the user asks a part of PDF, give clear and helpful information about it."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate(
    [
        ("system",system_prompt),
        ("human", "{input}"),
    ]
    )

In [33]:
from langchain.chains.combine_documents import create_stuff_documents_chain

create_stuff_documents_chain = create_stuff_documents_chain(model,prompt)


In [34]:
from langchain.chains import create_retrieval_chain

output = create_retrieval_chain(retriever, create_stuff_documents_chain)

In [36]:
output.invoke({"input":"What do u think these 2 articles"})['answer']

'The text you provided appears to be a snippet from a research paper on machine translation, specifically focusing on text-to-image generators and their performance across different languages. \n\nThe two main topics discussed are:\n\n1. **Initial Evaluation of Language Support in Text to Image Generators**: The author conducted an initial evaluation of how well text-to-image generators support different languages, using four languages: English, Spanish, Basque, and Latin. They used five text prompts to test the generators.\n\n2. **Options to Improve Performance**: The author briefly discusses options to make the performance of text-to-image generators consistent across many languages, although the details of this discussion are not provided in the snippet.\n\nOverall, the paper seems to be exploring the capabilities and limitations of text-to-image generators in supporting multiple languages, which is an important aspect of natural language processing and machine learning research.'